# Notes

Prerequisites: upload *data.zip* to Colab, or to the notebook directory.

The code is adopted from the Ruben Winastwan's article ([link](https://towardsdatascience.com/named-entity-recognition-with-bert-in-pytorch-a454405e0b6a)).

# Imports

In [1]:
%%capture
pip install transformers

In [2]:
import os
import glob

import torch
import pandas as pd
import numpy as np

from tqdm import tqdm
from torch.optim import SGD
from torch.utils.data import DataLoader
from transformers import BertTokenizerFast, BertForTokenClassification

# Data Preprocessing

In [3]:
def label(text: str, label: str = 'O', *, entity: bool):
    tokens = []
    labels = []
    if entity:
        label = f'B-{label}'
    for token in text.split():
        tokens.append(token)
        labels.append(label)
        if entity:
            label = 'I' + label[1:]
    return tokens, labels


def preprocess_file(text: str, ann_lines: list[str], path):
    entities = []
    for ann_line in ann_lines:
        # deconstruct the entity record into parts
        entity_raw, entity_text = ann_line.split('\t')[1], ann_line.split('\t')[2]
        entity_raw_split = entity_raw.split()
        entity_label, entity_start = entity_raw_split[0:2]
        entity_end = entity_raw_split[2] if len(entity_raw_split) == 3 else entity_raw_split[3]
        
        entity = {
            'text': entity_text,
            'label': entity_label,
            'start': int(entity_start),
            'end': int(entity_end),
        }
        entities.append(entity)

    tokens = []
    labels = []
    for i, entity in enumerate(entities):
        # label the non-entitiy tokens preceding the current entity as 'O'
        if i == 0 and entity['start'] > 0:
            no_entity_start = 0
        else:
            no_entity_start = entities[i - 1]['end'] + 1
        no_entity_end = entity['start']
        no_entity_text = text[no_entity_start : no_entity_end]
        no_entity_tokens, no_entity_labels = label(no_entity_text, entity=False)
        tokens.extend(no_entity_tokens)
        labels.extend(no_entity_labels)

        # label the entity tokens
        entity_text = text[entity['start'] : entity['end']]
        entity_tokens, entity_labels = label(entity_text, entity=True, label=entity['label'])

        # extend the lists
        tokens.extend(entity_tokens)
        labels.extend(entity_labels)

    return tokens, labels
            

def preprocess_dir(dir_path: str):
    tokens, labels, paths = [], [], []
    for txt_path in glob.glob(f'{dir_path}/*.txt'):
        with open(txt_path, mode='r') as txt:
            text = txt.read()
        
        with open(txt_path[:-3] + 'ann', mode='r') as ann:
            ann_lines = [line.rstrip() for line in ann if line[0] == 'T']

        file_tokens, file_labels = preprocess_file(text, ann_lines, txt_path)
        tokens.append(file_tokens)
        labels.append(file_labels)
        paths.append(txt_path)

    return tokens, labels, paths


def preprocess_dataset(path: str) -> pd.DataFrame:
    tokens_lists, labels_lists, paths = [], [], []
    for subdir in os.listdir(path):
        subdir_path = os.path.join(path, subdir)
        if not os.path.isdir(subdir_path):
            continue

        subdir_tokens, subdir_labels, subdir_paths = preprocess_dir(subdir_path)
        tokens_lists.extend(subdir_tokens)
        labels_lists.extend(subdir_labels)
        paths.extend(subdir_paths)

    texts = [' '.join(token_list) for token_list in tokens_lists]
    labels = [' '.join(labels_list) for labels_list in labels_lists]
    content = {
        'tokens_list': tokens_lists,
        'labels_list': labels_lists,
        'paths_list': paths,
        'text': texts,
        'labels': labels,
    }

    df_res = pd.DataFrame(content)

    return df_res

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
df = preprocess_dataset('/content/drive/MyDrive/Studying/NLP/train_data')
df

,tokens_list,labels_list,paths_list,text,labels
0,"[Пулеметы,, автоматы, и, снайперские, винтовки...","[O, O, O, O, O, O, O, O, B-NATIONALITY, O, O, ...",/content/drive/MyDrive/Studying/NLP/train_data...,"Пулеметы, автоматы и снайперские винтовки изъя...",O O O O O O O O B-NATIONALITY O O B-CITY O O O...
1,"[Российские, менеджеры, отказались, возглавлят...","[B-COUNTRY, O, O, O, O, O, B-ORGANIZATION, O, ...",/content/drive/MyDrive/Studying/NLP/train_data...,"Российские менеджеры отказались возглавлять ""г...",B-COUNTRY O O O O O B-ORGANIZATION O O O O O B...
2,"[Экс-претендент, на, губернаторский, пост, ста...","[O, O, O, O, O, B-PROFESSION, I-PROFESSION, O,...",/content/drive/MyDrive/Studying/NLP/train_data...,Экс-претендент на губернаторский пост стал и.о...,O O O O O B-PROFESSION I-PROFESSION O O O B-PR...
3,"[Марио, Монти, подаст, в, отставку, с, должнос...","[B-PERSON, I-PERSON, O, O, O, O, O, O, O, O, B...",/content/drive/MyDrive/Studying/NLP/train_data...,Марио Монти подаст в отставку с должности прем...,B-PERSON I-PERSON O O O O O O O O B-ORGANIZATI...
4,"[Россия, задерживает, поставку, генсека, Сегод...","[B-COUNTRY, O, O, O, O, O, B-CITY, O, O, O, O,...",/content/drive/MyDrive/Studying/NLP/train_data...,Россия задерживает поставку генсека Сегодня в ...,B-COUNTRY O O O O O B-CITY O O O O O O B-ORGAN...
...,...,...,...,...,...
836,"[Ларри, Пейдж, и, Сергей, Брин, перестали, воз...","[B-PERSON, I-PERSON, O, B-PERSON, I-PERSON, O,...",/content/drive/MyDrive/Studying/NLP/train_data...,Ларри Пейдж и Сергей Брин перестали возглавлят...,B-PERSON I-PERSON O B-PERSON I-PERSON O O B-OR...
837,"[Премьер-министр, Словении, Марьян, Шарец, под...","[B-PROFESSION, O, B-PERSON, I-PERSON, O, O, O,...",/content/drive/MyDrive/Studying/NLP/train_data...,Премьер-министр Словении Марьян Шарец подал в ...,B-PROFESSION O B-PERSON I-PERSON O O O B-PERSO...
838,"[Ушёл, из, жизни, Виктор, Васильевич, Тихонов,...","[O, O, O, B-PERSON, I-PERSON, I-PERSON, O, B-C...",/content/drive/MyDrive/Studying/NLP/train_data...,Ушёл из жизни Виктор Васильевич Тихонов В Моск...,O O O B-PERSON I-PERSON I-PERSON O B-CITY O O ...
839,"[Умерла, Людмила, Зыкина., Людмила, Зыкина, ср...","[O, O, O, O, O, O, O, O, O, B-AGE, B-DATE, O, ...",/content/drive/MyDrive/Studying/NLP/train_data...,Умерла Людмила Зыкина. Людмила Зыкина среди др...,O O O O O O O O O B-AGE B-DATE O B-CITY B-AGE ...


In [12]:
df.iloc[0].text

'Пулеметы, автоматы и снайперские винтовки изъяты в арендуемом американцами доме в Бишкеке 05/08/2008 10:35 БИШКЕК, 5 августа /Новости-Грузия/. Правоохранительные органы Киргизии Киргизии обнаружили в доме, арендуемом гражданами США в Бишкеке пулеметы, автоматы и снайперские винтовки, сообщает во вторник пресс-служба МВД Киргизии. "В ходе проведения оперативно-профилактического мероприятия под кодовым названием "Арсенал" в новостройке Ынтымак в доме, принадлежащем 66-летнему гражданину Киргизии и арендуемом гражданами США обнаружены и изъяты: шесть крупнокалиберных пулеметов с оптическим прицелом и с приборами ночного видения, 26 автоматов калибра 5,56 миллиметра два винчестера марки МОСВЕГА 12-го калибра, четыре ствола от крупнокалиберного пулемета, два подствольных гранатомета, четыре снайперские винтовки с оптическим прицелом защитного цвета, шесть пистолетов калибра 9 миллиметров марки Беретта одна винтовка", - говорится в сообщении МВД Пресс-служба отмечает, что на момент обыска "

In [13]:
idx = 3
df.iloc[idx]['text']

'Марио Монти подаст в отставку с должности премьер-министра Италии Фото: Reuters Фото: Reuters Фото: Reuters Фото: Reuters Премьер-министр Италии Марио Монти подтвердил намерение подать прошение об отставке после того, как парламент одобрит закон о государственном бюджете на 2013 г Этот и другие вопросы глава кабинета обсудил в ходе двухчасовой беседы с Президентом страны Джорджо Наполитано передает Reuters Отметим, что ранее входившая в парламентскую коалицию партия " Народ свободы отказала Монти в поддержке. В свою очередь 76-летний Сильвио Берлускони который является лидером указанной партии, за несколько часов до заявления Монти сообщил прессе, что сохраняет премьерские амбиции и готов вновь бороться за пост главы правительства республики в 2013 году В случае если бюджет будет принят "быстро", Монти готов немедленно сложить с себя полномочия. Примечательно, что Президент имеет полномочия объявить о роспуске парламента и проведении досрочных выборов. Несмотря на то, что такой вариан

In [14]:
df.iloc[idx]['labels']

'B-PERSON I-PERSON O O O O O O O O B-ORGANIZATION O B-ORGANIZATION O B-ORGANIZATION O B-ORGANIZATION O O B-PERSON I-PERSON O O O O O O O O O B-ORGANIZATION O O O O O B-DATE I-DATE I-DATE O O O O B-PROFESSION I-PROFESSION O O O O O O B-PROFESSION I-PROFESSION B-PERSON I-PERSON O B-ORGANIZATION O O O O O O O O O B-ORGANIZATION I-ORGANIZATION O B-PERSON O O O O O B-AGE B-PERSON I-PERSON O O O O O B-TIME I-TIME I-TIME O O B-PERSON O O O O O O O O O O O O B-PROFESSION I-PROFESSION I-PROFESSION B-DATE I-DATE I-DATE O O O O O O O B-PERSON O O O O O O O O B-PROFESSION O O O O O O O O O O O O O O O O O O B-COUNTRY O O O O O O O O O O O O B-PERSON O O O O O O B-ORGANIZATION I-ORGANIZATION O O O O O O B-PERSON O B-DATE I-DATE I-DATE O O O O B-PROFESSION I-PROFESSION B-PERSON O O O B-DATE I-DATE I-DATE I-DATE O O O O O O O O O O O O O O O O O O O O O B-DATE I-DATE I-DATE O O B-PERSON O B-DATE O O O O O O O O O O O B-DATE I-DATE I-DATE O O O O O O O O O O O O O B-DATE I-DATE I-DATE O O O O O O O O 

# Initialize Tokenizer

In [8]:
tokenizer = BertTokenizerFast.from_pretrained('bert-base-multilingual-cased')

# Create Dataset Class 

In [9]:
label_all_tokens = False

def align_label(texts, labels):
    tokenized_inputs = tokenizer(texts, padding='max_length', max_length=512, truncation=True)

    word_ids = tokenized_inputs.word_ids()

    previous_word_idx = None
    label_ids = []

    for word_idx in word_ids:

        if word_idx is None:
            label_ids.append(-100)

        elif word_idx != previous_word_idx:
            try:
                label_ids.append(labels_to_ids[labels[word_idx]])
            except:
                label_ids.append(-100)
        else:
            try:
                label_ids.append(labels_to_ids[labels[word_idx]] if label_all_tokens else -100)
            except:
                label_ids.append(-100)
        previous_word_idx = word_idx

    return label_ids

class DataSequence(torch.utils.data.Dataset):

    def __init__(self, df):

        lb = [i.split() for i in df['labels'].values.tolist()]
        txt = df['text'].values.tolist()
        self.texts = [tokenizer(str(i),
                               padding='max_length', max_length = 512, truncation=True, return_tensors="pt") for i in txt]
        self.labels = [align_label(i,j) for i,j in zip(txt, lb)]

    def __len__(self):

        return len(self.labels)

    def get_batch_data(self, idx):

        return self.texts[idx]

    def get_batch_labels(self, idx):

        return torch.LongTensor(self.labels[idx])

    def __getitem__(self, idx):

        batch_data = self.get_batch_data(idx)
        batch_labels = self.get_batch_labels(idx)

        return batch_data, batch_labels

# Split Data and Define Unique Labels

In [10]:
# df = df[0:1000]

labels = [i.split() for i in df['labels'].values.tolist()]
unique_labels = set()

for lb in labels:
        [unique_labels.add(i) for i in lb if i not in unique_labels]
labels_to_ids = {k: v for v, k in enumerate(unique_labels)}
ids_to_labels = {v: k for v, k in enumerate(unique_labels)}

df_train, df_val, df_test = np.split(df.sample(frac=1, random_state=42),
                            [int(.8 * len(df)), int(.9 * len(df))])

# Build Model

In [52]:
class BertModel(torch.nn.Module):

    def __init__(self):

        super(BertModel, self).__init__()

        self.bert = BertForTokenClassification.from_pretrained('bert-base-multilingual-cased',
                                                               num_labels=len(unique_labels))
    

    def forward(self, input_id, mask, label):

        output = self.bert(input_ids=input_id, attention_mask=mask, labels=label, return_dict=False)

        return output

# Model Training

In [82]:
model = BertModel()
model = model.to('cuda')

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForTokenClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint at 

In [98]:
def train_loop(model, df_train, df_val):

    train_dataset = DataSequence(df_train)
    val_dataset = DataSequence(df_val)

    train_dataloader = DataLoader(train_dataset, num_workers=4, batch_size=BATCH_SIZE, shuffle=True)
    val_dataloader = DataLoader(val_dataset, num_workers=4, batch_size=BATCH_SIZE)

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    optimizer = SGD(model.parameters(), lr=LEARNING_RATE)

    if use_cuda:
        model = model.cuda()

    best_acc = 0
    best_loss = 1000

    for epoch_num in range(EPOCHS):

        total_acc_train = 0
        total_loss_train = 0

        model.train()

        for train_data, train_label in tqdm(train_dataloader):

            train_label = train_label.to(device)
            mask = train_data['attention_mask'].squeeze(1).to(device)
            input_id = train_data['input_ids'].squeeze(1).to(device)

            optimizer.zero_grad()
            loss, logits = model(input_id, mask, train_label)

            for i in range(logits.shape[0]):
              logits_clean = logits[i][train_label[i] != -100]
              label_clean = train_label[i][train_label[i] != -100]
                

              predictions = logits_clean.argmax(dim=1)
              acc = (predictions == label_clean).float().mean()
              total_acc_train += acc
              total_loss_train += loss.item()

            loss.backward()
            optimizer.step()

        model.eval()

        total_acc_val = 0
        total_loss_val = 0

        for val_data, val_label in val_dataloader:

            val_label = val_label.to(device)
            mask = val_data['attention_mask'].squeeze(1).to(device)
            input_id = val_data['input_ids'].squeeze(1).to(device)

            loss, logits = model(input_id, mask, val_label)

            for i in range(logits.shape[0]):

              logits_clean = logits[i][val_label[i] != -100]
              label_clean = val_label[i][val_label[i] != -100]

              predictions = logits_clean.argmax(dim=1)
              acc = (predictions == label_clean).float().mean()
              total_acc_val += acc
              total_loss_val += loss.item()

        val_accuracy = total_acc_val / len(df_val)
        val_loss = total_loss_val / len(df_val)

        print(
            f'Epochs: {epoch_num + 1} | Loss: {total_loss_train / len(df_train): .3f} | Accuracy: {total_acc_train / len(df_train): .3f} | Val_Loss: {total_loss_val / len(df_val): .3f} | Accuracy: {total_acc_val / len(df_val): .3f}')

LEARNING_RATE = 2e-5
EPOCHS = 3
BATCH_SIZE = 2

# model = BertModel()
# model = model.to('cuda')
train_loop(model, df_train, df_val)

100%|██████████| 336/336 [01:04<00:00,  5.17it/s]


Epochs: 1 | Loss:  2.945 | Accuracy:  0.670 | Val_Loss:  2.232 | Accuracy:  0.761


100%|██████████| 336/336 [01:06<00:00,  5.04it/s]


Epochs: 2 | Loss:  1.986 | Accuracy:  0.747 | Val_Loss:  1.624 | Accuracy:  0.761


100%|██████████| 336/336 [01:06<00:00,  5.04it/s]


Epochs: 3 | Loss:  1.674 | Accuracy:  0.747 | Val_Loss:  1.524 | Accuracy:  0.761


In [ ]:
torch.save(model.state_dict(), f'/content/drive/MyDrive/Studying/NLP/models/model.pt')
# model.load_state_dict(torch.load(f'./models/best_inception.pt', map_location=device))

# Evaluate Model

In [21]:
def evaluate(model, df_test):

    test_dataset = DataSequence(df_test)

    test_dataloader = DataLoader(test_dataset, num_workers=4, batch_size=1)

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    if use_cuda:
        model = model.cuda()

    total_acc_test = 0.0

    for test_data, test_label in test_dataloader:

            test_label = test_label.to(device)
            mask = test_data['attention_mask'].squeeze(1).to(device)

            input_id = test_data['input_ids'].squeeze(1).to(device)

            loss, logits = model(input_id, mask, test_label)

            for i in range(logits.shape[0]):

              logits_clean = logits[i][test_label[i] != -100]
              label_clean = test_label[i][test_label[i] != -100]

              predictions = logits_clean.argmax(dim=1)
              acc = (predictions == label_clean).float().mean()
              total_acc_test += acc

    val_accuracy = total_acc_test / len(df_test)
    print(f'Test Accuracy: {total_acc_test / len(df_test): .3f}')

In [22]:
evaluate(model, df_test)

Test Accuracy:  0.749


In [13]:
test_dataset = DataSequence(df_test)

test_dataloader = DataLoader(test_dataset, num_workers=4, batch_size=1, shuffle=True)

use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

if use_cuda:
    model = model.cuda()

total_acc_test = 0.0

/usr/local/lib/python3.9/dist-packages/torch/utils/data/dataloader.py:561: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [99]:
for test_data, test_label in test_dataloader:

    test_label = test_label.to(device)
    mask = test_data['attention_mask'].squeeze(1).to(device)

    input_id = test_data['input_ids'].squeeze(1).to(device)

    loss, logits = model(input_id, mask, test_label)

    # break

    for i in range(logits.shape[0]):

        logits_clean = logits[i][test_label[i] != -100]
        label_clean = test_label[i][test_label[i] != -100]

        predictions = logits_clean.argmax(dim=1)
        acc = (predictions == label_clean).float().mean()
        total_acc_test += acc

    break

In [100]:
label_clean

tensor([ 0,  2,  0,  0,  2,  6,  0,  0,  0, 17, 57,  2,  6,  0, 34,  0,  0,  0,
         0,  0, 13,  2,  6,  0,  0,  2,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  2,  6,  0,
         0, 51,  0,  0, 51,  0,  2,  0, 24, 18,  0,  2,  0, 24, 18,  0,  2,  0,
        24, 18,  0,  2,  0, 24, 18,  0,  2,  0,  0,  0,  0,  2,  0, 24, 18,  0,
         2,  6,  0,  0, 13, 17, 57, 24, 18, 18, 18,  0,  0,  0,  0,  2,  0,  0,
         0,  0, 16, 40, 40, 24, 18, 18, 18, 18, 18,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0, 24, 18, 13,  4,  4,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0, 

In [101]:
predictions

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [32]:
logits_clean

tensor([[ 0.5190,  1.1520,  2.1227,  ...,  1.2356, -0.9892,  2.8318],
        [ 0.3606,  0.9914,  2.8317,  ...,  0.8318, -1.0038,  2.9986],
        [ 0.4556,  1.2642,  2.3142,  ...,  1.0828, -0.7622,  2.8795],
        ...,
        [ 0.3384,  0.5579,  2.4275,  ...,  0.9323, -0.9367,  2.7889],
        [ 0.6789,  1.1743,  2.2221,  ...,  0.8557, -0.8733,  2.7836],
        [ 0.1640,  1.2163,  2.6203,  ...,  1.0021, -1.0001,  3.2651]],
       device='cuda:0', grad_fn=<IndexBackward0>)

# Predict One Sentence

In [30]:
def align_word_ids(texts):
  
    tokenized_inputs = tokenizer(texts, padding='max_length', max_length=512, truncation=True)

    word_ids = tokenized_inputs.word_ids()

    previous_word_idx = None
    label_ids = []

    for word_idx in word_ids:

        if word_idx is None:
            label_ids.append(-100)

        elif word_idx != previous_word_idx:
            try:
                label_ids.append(1)
            except:
                label_ids.append(-100)
        else:
            try:
                label_ids.append(1 if label_all_tokens else -100)
            except:
                label_ids.append(-100)
        previous_word_idx = word_idx

    return label_ids


def evaluate_one_text(model, sentence):


    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    if use_cuda:
        model = model.cuda()

    text = tokenizer(sentence, padding='max_length', max_length = 512, truncation=True, return_tensors="pt")

    mask = text['attention_mask'].to(device)
    input_id = text['input_ids'].to(device)
    label_ids = torch.Tensor(align_word_ids(sentence)).unsqueeze(0).to(device)

    logits = model(input_id, mask, None)
    logits_clean = logits[0][label_ids != -100]

    predictions = logits_clean.argmax(dim=1).tolist()
    prediction_label = [ids_to_labels[i] for i in predictions]
    print(sentence)
    print(prediction_label)

In [ ]:
text = 'Глава департамента ЦБ РФ Надежда Иванова получила статус зампреда\
\
Иванова, которой 13 июня исполнилось 60 лет, всю свою жизнь проработала в системе ЦБ. Сводный экономический департамент Банка России возглавляет с 1995 года.\
Здание Центрального банка РФ. Архив\
\
Директор сводного экономического департамента Банка России Надежда Иванова назначена также на должность заместителя председателя ЦБ, сообщил в четверг регулятор.'

evaluate_one_text(model, text

Глава департамента ЦБ РФ Надежда Иванова получила статус зампредаИванова, которой 13 июня исполнилось 60 лет, всю свою жизнь проработала в системе ЦБ. Сводный экономический департамент Банка России возглавляет с 1995 года.Здание Центрального банка РФ. АрхивДиректор сводного экономического департамента Банка России Надежда Иванова назначена также на должность заместителя председателя ЦБ, сообщил в четверг регулятор.
['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']


In [ ]:
evaluate_one_text(model, df.iloc[3]['text'])

Марио Монти подаст в отставку с должности премьер-министра Италии Фото: Reuters Фото: Reuters Фото: Reuters Фото: Reuters Премьер-министр Италии Марио Монти подтвердил намерение подать прошение об отставке после того, как парламент одобрит закон о государственном бюджете на 2013 г Этот и другие вопросы глава кабинета обсудил в ходе двухчасовой беседы с Президентом страны Джорджо Наполитано передает Reuters Отметим, что ранее входившая в парламентскую коалицию партия " Народ свободы отказала Монти в поддержке. В свою очередь 76-летний Сильвио Берлускони который является лидером указанной партии, за несколько часов до заявления Монти сообщил прессе, что сохраняет премьерские амбиции и готов вновь бороться за пост главы правительства республики в 2013 году В случае если бюджет будет принят "быстро", Монти готов немедленно сложить с себя полномочия. Примечательно, что Президент имеет полномочия объявить о роспуске парламента и проведении досрочных выборов. Несмотря на то, что такой вариант